In [17]:
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

In [18]:
# Input data
df = pd.read_csv('../Output/tweets1.csv')

In [19]:
# Drop Null values from data frame
df = df.dropna(subset=['Tokenized'])

In [20]:
# Create tfidf scores and fit the data to the transformer

from sklearn.feature_extraction.text import TfidfVectorizer  
tfidfconverter = TfidfVectorizer(max_features=750, analyzer = 'word')  

tf_transformer = tfidfconverter.fit(df['Tokenized'])
X = tfidfconverter.fit_transform(df['Tokenized'])

df['tfid'] = X
df.head()

,ItemID,Sentiment,Tokenized,tfid
0,1,0,sad friend,"(0, 514)\t0.6902929415704523\n (0, 251)\t0...."
1,2,0,miss new moon trailer,"(0, 514)\t0.6902929415704523\n (0, 251)\t0...."
2,3,1,already,"(0, 514)\t0.6902929415704523\n (0, 251)\t0...."
3,4,0,cry dentist since get crown put,"(0, 514)\t0.6902929415704523\n (0, 251)\t0...."
4,5,0,think mi cheating,"(0, 514)\t0.6902929415704523\n (0, 251)\t0...."


In [21]:
# Test, Train, Split
X_train, X_test, y_train, y_test = train_test_split(
    X, df['Sentiment'], random_state=1)

In [22]:
# Fit the tfidf data to a Gaussian Naive Bayes model

from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
model = gnb.fit(X_train.toarray(), y_train)
y_pred = model.predict(X_test.toarray())


In [23]:
# Output results of model

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

[[6712 3816]
 [3784 9947]]
              precision    recall  f1-score   support

           0       0.64      0.64      0.64     10528
           1       0.72      0.72      0.72     13731

   micro avg       0.69      0.69      0.69     24259
   macro avg       0.68      0.68      0.68     24259
weighted avg       0.69      0.69      0.69     24259

0.6867142091594872


In [24]:
# Save the model and prepare the tfidf vocabulary for the "new" tweets that will be scraped

from sklearn.feature_extraction.text import TfidfVectorizer
import pickle

# Dump the file
pickle.dump(tf_transformer, open("tfidf1.pkl", "wb"))

# Testing phase
tf1 = pickle.load(open("tfidf1.pkl", 'rb'))

# Create new tfidfVectorizer with old vocabulary for new tweets that weill be scraped
tf1_new = TfidfVectorizer(analyzer='word', ngram_range=(1,2), stop_words = "english", lowercase = True,
                          max_features = 500000, vocabulary = tf1.vocabulary_)

# Input to fix error below
input=[input]

# Input fully cleansed tweets into the following "fit_transform" function to get appropriate tfidf score
X_tf1 = tf1_new.fit_transform('new tweet data will go here')
print(X_tf1)

# Predict sentiment of new tweets
model.predict(X_tf1)

ValueError: Iterable over raw text documents expected, string object received.